In [1]:
#%%
import pandas as pd
import numpy as np
import pickle
import os, sys
from pathlib import Path
import argparse


PROJ_PATH = Path('/home/dogu86/lung_synthesis')
sys.path.append(PROJ_PATH.joinpath('src').as_posix())

from MyModule.utils import *
config = load_config()
INPUT_PATH = Path('/mnt/synthetic_data/lung/data/processed/2_restore/restore_to_db_form')
OUTPUT_PATH = Path('/mnt/synthetic_data/lung/data/processed/3_postprocess')

if not OUTPUT_PATH.exists() :
    OUTPUT_PATH.mkdir(parents=True)

file_name = "LUNG_TRTM_CASB"

# %%
def load_file_epsilon(epsilon):
    return read_file(INPUT_PATH, f'{file_name}_{epsilon}.pkl')

In [59]:
data = load_file_epsilon(1000)

In [60]:
data = load_file_epsilon(1000)
data = data.rename(columns = {'TIME':'CSTR_STRT_YMD'})
data = data.dropna(subset=['CSTR_CLSF_NM', 'CSTR_PRPS_CD'], how='any')
data = data.drop(data[data['CSTR_STRT_YMD']=='2222-02-22'].index)
data['CSTR_END_YMD'] = data['CSTR_STRT_YMD'] + pd.to_timedelta(28, 'days') # need to check quantity of additonal
data['CSTR_CYCL_VL'] = data.groupby(['PT_SBST_NO','CSTR_PRPS_CD']).cumcount()+1

grouped = data.groupby(['PT_SBST_NO', 'CSTR_PRPS_CD'])

In [61]:
data['CSTR_STRT_YMD'] = grouped['CSTR_END_YMD'].shift(1)
#data['CSTR_STRT_YMD'] = grouped['CSTR_STRT_YMD'].apply(lambda x: x.fillna('2222-02-22'))

In [62]:
data[data['PT_SBST_NO']=='9999']

,PT_SBST_NO,CSTR_STRT_YMD,CSTR_CLSF_NM,CSTR_PRPS_CD,CSTR_CYCL_VL,CSTR_END_YMD
425571,9999,NaT,미수술 항암화학요법,3.0,1,2223-12-14
425576,9999,2223-12-14,미수술 항암화학요법,3.0,2,2224-09-25
425578,9999,2224-09-25,미수술 항암화학요법,3.0,3,2225-02-20
425579,9999,2225-02-20,미수술 항암화학요법,3.0,4,2225-05-15
425580,9999,2225-05-15,미수술 항암화학요법,3.0,5,2225-08-12
425582,9999,2225-08-12,미수술 항암화학요법,3.0,6,2226-01-28
425583,9999,2226-01-28,미수술 항암화학요법,3.0,7,2226-02-20


In [32]:
original_path = PROJ_PATH.joinpath(f'data/raw/{file_name}.xlsx')
original_data = pd.read_excel(original_path)
head = pd.read_excel(original_path, nrows=0)

data = pd.concat([head, data])

reference1 = original_data[['CSTR_PRPS_CD','CSTR_PRPS_NM']].drop_duplicates().reset_index(drop=True)
data = data.drop(columns='CSTR_PRPS_NM')
data = data.merge(reference1, how='left', on='CSTR_PRPS_CD')

data['CENTER_CD'] = 00000
data['IRB_APRV_NO'] = '2-2222-02-22'
data['CRTN_DT'] = '0200.0'
data['CSTR_SEQ'] = 1

dtypes = head.dtypes.to_dict()
data = data.astype(dtypes)

data['CSTR_STRT_YMD']= data['CSTR_STRT_YMD'].astype('datetime64[ns]')
data['CSTR_END_YMD']= data['CSTR_END_YMD'].astype('datetime64[ns]')

data = data[head.columns]

/tmp/ipykernel_25535/712378079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CSTR_STRT_YMD'] = grouped['CSTR_END_YMD'].shift(1)
/tmp/ipykernel_25535/712378079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CSTR_STRT_YMD'] = grouped['CSTR_STRT_YMD'].apply(lambda x: x.fillna('2222-02-22'))


In [34]:
data[data['PT_SBST_NO']=='9999']

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,CSTR_STRT_YMD,CSTR_SEQ,CSTR_CLSF_NM,CSTR_END_YMD,CSTR_PRPS_CD,CSTR_PRPS_NM,CSTR_CYCL_VL,CRTN_DT
223519,0,2-2222-02-22,9999,2223-12-14,1,미수술 항암화학요법,2224-09-25,3.0,Palliative,2,0200.0
223520,0,2-2222-02-22,9999,2224-09-25,1,미수술 항암화학요법,2225-02-20,3.0,Palliative,3,0200.0
223521,0,2-2222-02-22,9999,2225-02-20,1,미수술 항암화학요법,2225-05-15,3.0,Palliative,4,0200.0
223522,0,2-2222-02-22,9999,2225-05-15,1,미수술 항암화학요법,2225-08-12,3.0,Palliative,5,0200.0
223523,0,2-2222-02-22,9999,2225-08-12,1,미수술 항암화학요법,2226-01-28,3.0,Palliative,6,0200.0
223524,0,2-2222-02-22,9999,2226-01-28,1,미수술 항암화학요법,2226-02-20,3.0,Palliative,7,0200.0


In [31]:
data = data.dropna(subset=data.columns[1:-1],how='any')
data = data.drop('BSPT_FRST_DIAG_YMD',axis=1)
data = data.rename(columns={'TIME':"BSPT_FRST_DIAG_YMD"})

KeyError: "['BSPT_FRST_DIAG_YMD'] not found in axis"

In [71]:
original_path = PROJ_PATH.joinpath(f'/home/dogu86/lung_synthesis/data/raw/{file_name}.xlsx')
original_data = pd.read_excel(original_path)
head = pd.read_excel(original_path, nrows = 0)
data['BSPT_BRYM'] = data['BSPT_FRST_DIAG_YMD'] - pd.to_timedelta(data['BSPT_AGE']*365, 'days')
data

,PT_SBST_NO,BSPT_FRST_DIAG_YMD,BSPT_SEX_CD,BSPT_FRST_DIAG_CD,BSPT_STAG_VL,BSPT_T_STAG_VL,BSPT_N_STAG_VL,BSPT_M_STAG_VL,BSPT_AGE,BSPT_BRYM
0,0,2222-02-22,F,C343,2.0,2.0,0.0,0.0,43.0,2179-03-04
1,0,2222-02-25,F,C343,2.0,2.0,0.0,0.0,43.0,2179-03-07
2,0,2225-05-06,F,C343,2.0,2.0,0.0,0.0,43.0,2182-05-16
3,0,2225-05-27,F,C343,2.0,2.0,0.0,0.0,43.0,2182-06-06
4,0,2225-07-08,F,C343,2.0,2.0,0.0,0.0,43.0,2182-07-18
...,...,...,...,...,...,...,...,...,...,...
425579,9999,2225-04-17,F,C341,1.0,1.0,0.0,0.0,79.0,2146-05-06
425580,9999,2225-07-15,F,C341,1.0,1.0,0.0,0.0,79.0,2146-08-03
425581,9999,2225-10-07,F,C341,1.0,1.0,0.0,0.0,79.0,2146-10-26
425582,9999,2225-12-31,F,C341,1.0,1.0,0.0,0.0,79.0,2147-01-19


In [72]:
epsilon = 1000
dead = pd.read_excel(f'/mnt/synthetic_data/lung/data/processed/3_postprocess/LUNG_DEAD_NFRM_{epsilon}.xlsx')
dead['PT_SBST_NO'] = dead['PT_SBST_NO'].astype('str')
dead = dead.rename(columns={'DEAD_YMD':"BSPT_DEAD_YMD"})

data = pd.merge(data,dead[['PT_SBST_NO','BSPT_DEAD_YMD']],how='left',on='PT_SBST_NO')

oprt = pd.read_excel(f'/mnt/synthetic_data/lung/data/processed/3_postprocess/LUNG_OPRT_NFRM_{epsilon}.xlsx')
oprt['PT_SBST_NO'] = oprt['PT_SBST_NO'].astype('str')
oprt = oprt.drop_duplicates(subset='PT_SBST_NO')
oprt = oprt.rename(columns={"OPRT_YMD":"BSPT_FRST_OPRT_YMD"})

data = pd.merge(data,oprt[['PT_SBST_NO','BSPT_FRST_OPRT_YMD']],how='left',on='PT_SBST_NO')

rdt = pd.read_excel(f'/mnt/synthetic_data/lung/data/processed/3_postprocess/LUNG_TRTM_RD_{epsilon}.xlsx')
rdt['PT_SBST_NO'] = rdt['PT_SBST_NO'].astype('str')
rdt = rdt.rename(columns={"RDT_STRT_YMD":"BSPT_FRST_RDT_STRT_YMD"})

data = pd.merge(data,rdt[['PT_SBST_NO','BSPT_FRST_RDT_STRT_YMD']],how='left',on='PT_SBST_NO')

In [73]:
data = data.drop_duplicates(subset='PT_SBST_NO')

In [75]:
original_path = PROJ_PATH.joinpath(f'data/raw/{file_name}.xlsx')
original_data = pd.read_excel(original_path)
head = pd.read_excel(original_path, nrows=0)
original_data

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,BSPT_SEX_CD,BSPT_BRYM,BSPT_FRST_DIAG_YMD,BSPT_FRST_DIAG_CD,BSPT_FRST_DIAG_NM,BSPT_IDGN_AGE,BSPT_FRST_OPRT_YMD,...,BSPT_FRST_RDT_STRT_YMD,BSPT_DEAD_YMD,CENTER_LAST_VST_YMD,OVRL_SRVL_DTRN_DCNT,BSPT_STAG_CLSF_CD,BSPT_STAG_VL,BSPT_T_STAG_VL,BSPT_N_STAG_VL,BSPT_M_STAG_VL,CRTN_DT
0,90,4-2021-05-20,RN00075817,M,19420520,20190718,C343,"Lower lobe, bronchus or lung",77,NaN,...,NaN,20191012.0,20191012.0,86,c,4,x,x,1,2022-11-02
1,90,4-2021-05-20,RN00100695,M,19341014,20120503,C343,"Lower lobe, bronchus or lung",77,NaN,...,20120523.0,20150525.0,20150521.0,1117,c,3,3,2,0,2022-11-02
2,90,4-2021-05-20,RN00104968,F,19410405,20140520,C343,"Lower lobe, bronchus or lung",73,20140613.0,...,NaN,NaN,20190530.0,1836,p,1,2,0,0,2022-11-02
3,90,4-2021-05-20,RN00110825,F,19560813,20200605,C342,"Middle lobe, bronchus or lung",63,NaN,...,NaN,NaN,20210108.0,217,p,1,1,0,0,2022-11-02
4,90,4-2021-05-20,RN00083644,M,19541218,20150120,C341,"Upper lobe, bronchus or lung",60,NaN,...,NaN,NaN,20200710.0,1998,p,2,2,0,0,2022-11-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11143,90,4-2021-05-20,RN00092310,M,19480925,20090302,C348,Overlapping lesion of bronchus and lung,60,NaN,...,20090310.0,20090521.0,20090521.0,80,p,4,4,2,1,2022-11-02
11144,90,4-2021-05-20,RN00042625,M,19580816,20200706,C343,"Lower lobe, bronchus or lung",61,NaN,...,20200723.0,NaN,20210603.0,332,c,4,2,3,1,2022-11-02
11145,90,4-2021-05-20,RN00052813,F,19541104,20181220,C341,"Upper lobe, bronchus or lung",64,20190207.0,...,NaN,NaN,20210201.0,774,p,1,1,0,0,2022-11-02
11146,90,4-2021-05-20,RN00004040,F,19390403,20151204,C343,"Lower lobe, bronchus or lung",76,NaN,...,NaN,NaN,20160115.0,42,c,4,x,1,1,2022-11-02


In [76]:
data = pd.concat([head, data])

In [51]:
head

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,BSPT_SEX_CD,BSPT_BRYM,BSPT_FRST_DIAG_YMD,BSPT_FRST_DIAG_CD,BSPT_FRST_DIAG_NM,BSPT_IDGN_AGE,BSPT_FRST_OPRT_YMD,...,BSPT_FRST_RDT_STRT_YMD,BSPT_DEAD_YMD,CENTER_LAST_VST_YMD,OVRL_SRVL_DTRN_DCNT,BSPT_STAG_CLSF_CD,BSPT_STAG_VL,BSPT_T_STAG_VL,BSPT_N_STAG_VL,BSPT_M_STAG_VL,CRTN_DT


In [77]:
data['BSPT_FRST_ANCN_TRTM_STRT_YMD'] = '2222-02-22'
data['BSPT_IDGN_AGE'] = data['BSPT_AGE']

In [78]:
data['CENTER_CD'] = 00000
data['IRB_APRV_NO'] = '2-2222-02-22'
data['CRTN_DT'] = '0200.0'
data['BSPT_STAG_CLSF_CD'] = 'not synthesized'

In [40]:
data.columns

Index(['CENTER_CD', 'IRB_APRV_NO', 'PT_SBST_NO', 'BSPT_SEX_CD', 'BSPT_BRYM',
       'BSPT_FRST_DIAG_YMD', 'BSPT_FRST_DIAG_CD', 'BSPT_FRST_DIAG_NM',
       'BSPT_IDGN_AGE', 'BSPT_FRST_OPRT_YMD', 'BSPT_FRST_ANCN_TRTM_STRT_YMD',
       'BSPT_FRST_RDT_STRT_YMD', 'BSPT_DEAD_YMD', 'CENTER_LAST_VST_YMD',
       'OVRL_SRVL_DTRN_DCNT', 'BSPT_STAG_CLSF_CD', 'BSPT_STAG_VL',
       'BSPT_T_STAG_VL', 'BSPT_N_STAG_VL', 'BSPT_M_STAG_VL', 'CRTN_DT',
       'BSPT_AGE'],
      dtype='object')

In [79]:
reference1 = original_data[['BSPT_FRST_DIAG_CD','BSPT_FRST_DIAG_NM']].drop_duplicates().reset_index(drop=True)
head = pd.read_excel(original_path, nrows = 0)
head = head.drop(columns=["BSPT_FRST_DIAG_CD","BSPT_FRST_DIAG_NM"])
data = pd.concat([head,data])

data = data.drop(columns='BSPT_FRST_DIAG_NM')
data = data.merge(reference1, how='left', on='BSPT_FRST_DIAG_CD')

In [62]:
data

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,BSPT_SEX_CD,BSPT_BRYM,BSPT_FRST_DIAG_YMD,BSPT_IDGN_AGE,BSPT_FRST_OPRT_YMD,BSPT_FRST_ANCN_TRTM_STRT_YMD,BSPT_FRST_RDT_STRT_YMD,...,OVRL_SRVL_DTRN_DCNT,BSPT_STAG_CLSF_CD,BSPT_STAG_VL,BSPT_T_STAG_VL,BSPT_N_STAG_VL,BSPT_M_STAG_VL,CRTN_DT,BSPT_FRST_DIAG_CD,BSPT_AGE,BSPT_FRST_DIAG_NM
0,0,2-2222-02-22,0,F,2179-03-04 00:00:00,2222-02-22 00:00:00,43.0,NaT,2222-02-22,NaT,...,NaN,not synthesized,2.0,2.0,0.0,0.0,0200.0,C343,43.0,"Lower lobe, bronchus or lung"
1,0,2-2222-02-22,1,F,2156-03-09 00:00:00,2222-02-22 00:00:00,66.0,2228-02-12 00:00:00,2222-02-22,2229-12-10 00:00:00,...,NaN,not synthesized,4.0,2.0,0.0,1.0,0200.0,C341,66.0,"Upper lobe, bronchus or lung"
2,0,2-2222-02-22,10,M,2156-03-09 00:00:00,2222-02-22 00:00:00,66.0,NaT,2222-02-22,2223-03-20 00:00:00,...,NaN,not synthesized,4.0,x,x,1.0,0200.0,C341,66.0,"Upper lobe, bronchus or lung"
3,0,2-2222-02-22,100,M,2167-03-07 00:00:00,2222-02-22 00:00:00,55.0,NaT,2222-02-22,NaT,...,NaN,not synthesized,1.0,1.0,0.0,0.0,0200.0,C341,55.0,"Upper lobe, bronchus or lung"
4,0,2-2222-02-22,1000,M,2169-03-06 00:00:00,2222-02-22 00:00:00,53.0,NaT,2222-02-22,2222-09-18 00:00:00,...,NaN,not synthesized,4.0,2.0,2,1.0,0200.0,C341,53.0,"Upper lobe, bronchus or lung"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43995,0,2-2222-02-22,9995,F,2171-03-06 00:00:00,2222-02-22 00:00:00,51.0,NaT,2222-02-22,NaT,...,NaN,not synthesized,3.0,2.0,2,0.0,0200.0,C341,51.0,"Upper lobe, bronchus or lung"
43996,0,2-2222-02-22,9996,M,2158-03-09 00:00:00,2222-02-22 00:00:00,64.0,NaT,2222-02-22,NaT,...,NaN,not synthesized,3.0,x,3,0.0,0200.0,C343,64.0,"Lower lobe, bronchus or lung"
43997,0,2-2222-02-22,9997,F,2154-03-10 00:00:00,2222-02-22 00:00:00,68.0,NaT,2222-02-22,2222-04-28 00:00:00,...,NaN,not synthesized,4.0,4,0.0,1.0,0200.0,C343,68.0,"Lower lobe, bronchus or lung"
43998,0,2-2222-02-22,9998,M,2156-03-09 00:00:00,2222-02-22 00:00:00,66.0,NaT,2222-02-22,2222-02-22 00:00:00,...,NaN,not synthesized,4.0,2.0,2,1.0,0200.0,C341,66.0,"Upper lobe, bronchus or lung"


In [80]:
dtypes = head.dtypes.to_dict()
data = data.astype(dtypes)
data['BSPT_BRYM']= data['BSPT_BRYM'].astype('datetime64[ns]')
data['BSPT_FRST_DIAG_YMD']= data['BSPT_FRST_DIAG_YMD'].astype('datetime64[ns]')
data['BSPT_FRST_OPRT_YMD']= data['BSPT_FRST_OPRT_YMD'].astype('datetime64[ns]')
data['BSPT_FRST_RDT_STRT_YMD']= data['BSPT_FRST_RDT_STRT_YMD'].astype('datetime64[ns]')
data['BSPT_FRST_ANCN_TRTM_STRT_YMD']= data['BSPT_FRST_ANCN_TRTM_STRT_YMD'].astype('datetime64[ns]')
data['BSPT_DEAD_YMD']= data['BSPT_DEAD_YMD'].astype('datetime64[ns]')

data = data[original_data.columns]

In [82]:
data

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,BSPT_SEX_CD,BSPT_BRYM,BSPT_FRST_DIAG_YMD,BSPT_FRST_DIAG_CD,BSPT_FRST_DIAG_NM,BSPT_IDGN_AGE,BSPT_FRST_OPRT_YMD,...,BSPT_FRST_RDT_STRT_YMD,BSPT_DEAD_YMD,CENTER_LAST_VST_YMD,OVRL_SRVL_DTRN_DCNT,BSPT_STAG_CLSF_CD,BSPT_STAG_VL,BSPT_T_STAG_VL,BSPT_N_STAG_VL,BSPT_M_STAG_VL,CRTN_DT
0,0,2-2222-02-22,0,F,2179-03-04,2222-02-22,C343,"Lower lobe, bronchus or lung",43.0,NaT,...,NaT,2231-10-17,NaN,NaN,not synthesized,2.0,2.0,0.0,0.0,0200.0
1,0,2-2222-02-22,1,F,2156-03-09,2222-02-22,C341,"Upper lobe, bronchus or lung",66.0,2228-02-12,...,2229-12-10,NaT,NaN,NaN,not synthesized,4.0,2.0,0.0,1.0,0200.0
2,0,2-2222-02-22,10,M,2156-03-09,2222-02-22,C341,"Upper lobe, bronchus or lung",66.0,NaT,...,2223-03-20,2223-08-22,NaN,NaN,not synthesized,4.0,x,x,1.0,0200.0
3,0,2-2222-02-22,100,M,2167-03-07,2222-02-22,C341,"Upper lobe, bronchus or lung",55.0,NaT,...,NaT,NaT,NaN,NaN,not synthesized,1.0,1.0,0.0,0.0,0200.0
4,0,2-2222-02-22,1000,M,2169-03-06,2222-02-22,C341,"Upper lobe, bronchus or lung",53.0,NaT,...,2222-09-18,NaT,NaN,NaN,not synthesized,4.0,2.0,2,1.0,0200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43995,0,2-2222-02-22,9995,F,2171-03-06,2222-02-22,C341,"Upper lobe, bronchus or lung",51.0,NaT,...,NaT,NaT,NaN,NaN,not synthesized,3.0,2.0,2,0.0,0200.0
43996,0,2-2222-02-22,9996,M,2158-03-09,2222-02-22,C343,"Lower lobe, bronchus or lung",64.0,NaT,...,NaT,2224-03-05,NaN,NaN,not synthesized,3.0,x,3,0.0,0200.0
43997,0,2-2222-02-22,9997,F,2154-03-10,2222-02-22,C343,"Lower lobe, bronchus or lung",68.0,NaT,...,2222-04-28,2223-12-12,NaN,NaN,not synthesized,4.0,4,0.0,1.0,0200.0
43998,0,2-2222-02-22,9998,M,2156-03-09,2222-02-22,C341,"Upper lobe, bronchus or lung",66.0,NaT,...,2222-02-22,2227-09-25,NaN,NaN,not synthesized,4.0,2.0,2,1.0,0200.0


In [370]:
pd.read_excel(f'/mnt/synthetic_data/lung/data/processed/3_postprocess/LUNG_EX_PLMN_{0.1}.xlsx')

,Unnamed: 0,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,PLEX_YMD,PLEX_SEQ,PLEX_KIND_CD,PLEX_KIND_NM,PLEX_NM,PLEX_RSLT_VL,CRTN_DT
0,280,0,2-2222-02-22,10006,2222-02-22,0,1,호흡기능검사,FEV1_FVC_P,71.0,200.0
1,436,0,2-2222-02-22,10013,2222-02-22,0,1,호흡기능검사,FEV1_FVC_P,80.0,200.0
2,638,0,2-2222-02-22,10023,2222-02-22,0,1,호흡기능검사,FEV1_FVC_P,38.0,200.0
3,964,0,2-2222-02-22,10040,2222-02-22,0,1,호흡기능검사,FEV1_FVC_P,75.0,200.0
4,1288,0,2-2222-02-22,10054,2222-02-22,0,1,호흡기능검사,FEV1_FVC_P,65.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...
17185,2126043,0,2-2222-02-22,9914,2222-03-01,0,1,호흡기능검사,FVC_P,64.0,200.0
17186,2126127,0,2-2222-02-22,9919,2222-04-06,0,1,호흡기능검사,FVC_P,52.0,200.0
17187,2126856,0,2-2222-02-22,9952,2222-03-06,0,1,호흡기능검사,FVC_P,107.0,200.0
17188,2127273,0,2-2222-02-22,9972,2222-02-22,0,1,호흡기능검사,FVC_P,73.0,200.0


In [374]:
pd.read_excel(f'/mnt/synthetic_data/lung/data/processed/3_postprocess/LUNG_PTH_SRGC_{epsilon}.xlsx')

,Unnamed: 0,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,SGPT_ACPT_YMD,SGPT_SEQ,SGPT_READ_YMD,SGPT_LUNGM_SIZE_VL,SGPT_CELL_TYPE_CONT,SGPT_CELL_DIFF_CONT,...,SGPT_SFMG_VL,SGPT_LN_DSCT_CONT,SGPT_MTST_LN_CNT,SGPT_MTST_LMP_SITE_CONT,SGPT_PATL_STAG_VL,SGPT_PATL_ETC_STAG_VL,SGPT_PATL_T_STAG_VL,SGPT_PATL_N_STAG_VL,SGPT_PATL_M_STAG_VL,CRTN_DT
0,1119,0,2-2222-02-22,10048,2222-02-22,1,2222-02-25,5.6,Non small cell carcinoma,NaN,...,3.0cm,not synthesized,0.0,not synthesized,NaN,not synthesized,3,0,0,200.0
1,1127,0,2-2222-02-22,10048,2222-06-29,1,2222-07-02,NaN,NaN,NaN,...,NaN,not synthesized,NaN,not synthesized,NaN,not synthesized,NaN,NaN,NaN,200.0
2,1132,0,2-2222-02-22,10048,2223-06-13,1,2223-06-16,5.6,Non small cell carcinoma,NaN,...,3.0cm,not synthesized,0.0,not synthesized,NaN,not synthesized,3,0,0,200.0
3,2228,0,2-2222-02-22,10098,2228-10-20,1,2228-10-23,11.0,Non small cell carcinoma,NaN,...,2.5cm,not synthesized,0.0,not synthesized,NaN,not synthesized,3,0,NaN,200.0
4,3092,0,2-2222-02-22,10141,2227-01-31,1,2227-02-03,2.0,Other,NaN,...,NaN,not synthesized,0.0,not synthesized,NaN,not synthesized,2,0,0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3350,423924,0,2-2222-02-22,9923,2222-02-22,1,2222-02-25,NaN,NaN,NaN,...,NaN,not synthesized,NaN,not synthesized,NaN,not synthesized,3,NaN,NaN,200.0
3351,423934,0,2-2222-02-22,9923,2225-02-12,1,2225-02-15,1.4,Non small cell carcinoma,NaN,...,NaN,not synthesized,NaN,not synthesized,NaN,not synthesized,3,x,NaN,200.0
3352,424394,0,2-2222-02-22,9944,2222-04-05,1,2222-04-08,5.5,Non small cell carcinoma,NaN,...,1.2cm,not synthesized,1.0,not synthesized,NaN,not synthesized,2,1,0,200.0
3353,425155,0,2-2222-02-22,9982,2222-02-22,1,2222-02-25,0.9,Non small cell carcinoma,NaN,...,0.6cm,not synthesized,NaN,not synthesized,NaN,not synthesized,3,x,NaN,200.0


In [372]:
bpth

,Unnamed: 0,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,BPTH_ACPT_YMD,BPTH_SEQ,BPTH_READ_YMD,BPTH_SITE_CONT,BPTH_BPSY_MTHD_DETL_CONT,CRTN_DT
0,155,0,2-2222-02-22,10,2222-03-10,1,2222-03-13,Lymph node,non small cell lung cancer,200.0
1,195,0,2-2222-02-22,10000,2222-05-06,1,2222-05-09,Lymph node,non small cell lung cancer,200.0
2,208,0,2-2222-02-22,10001,2222-06-01,1,2222-06-04,Lymph node,non small cell lung cancer,200.0
3,245,0,2-2222-02-22,10003,2222-09-14,1,2222-09-17,Lung,non small cell lung cancer,200.0
4,297,0,2-2222-02-22,10007,2222-03-10,1,2222-03-13,Lung,non small cell lung cancer,200.0
...,...,...,...,...,...,...,...,...,...,...
14085,425215,0,2-2222-02-22,9985,2222-12-26,1,2222-12-29,Lung,non small cell lung cancer,200.0
14086,425414,0,2-2222-02-22,9991,2222-02-23,1,2222-02-26,Lung,small cell lung cancer,200.0
14087,425443,0,2-2222-02-22,9992,2225-02-12,1,2225-02-15,Lung,non small cell lung cancer,200.0
14088,425478,0,2-2222-02-22,9995,2222-03-31,1,2222-04-03,Lung,non small cell lung cancer,200.0


In [5]:
pd.read_excel('/mnt/synthetic_data/lung/data/processed/3_postprocess/LUNG_PT_BSNF_10.xlsx')

,Unnamed: 0,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,BSPT_SEX_CD,BSPT_BRYM,BSPT_FRST_DIAG_YMD,BSPT_IDGN_AGE,BSPT_FRST_OPRT_YMD,BSPT_FRST_ANCN_TRTM_STRT_YMD,BSPT_FRST_RDT_STRT_YMD,BSPT_DEAD_YMD,CENTER_LAST_VST_YMD,OVRL_SRVL_DTRN_DCNT,BSPT_STAG_CLSF_CD,BSPT_STAG_VL,BSPT_T_STAG_VL,BSPT_N_STAG_VL,BSPT_M_STAG_VL,CRTN_DT
0,0,0,2-2222-02-22,0,F,2179-03-04,2222-02-22,43,NaT,2222-02-22,NaT,2231-10-16,NaN,NaN,not synthesized,2,2,0,0,200.0
1,1,0,2-2222-02-22,1,F,2156-03-09,2222-02-22,66,2222-03-13,2222-02-22,2223-12-09,NaT,NaN,NaN,not synthesized,4,2,0,1,200.0
2,2,0,2-2222-02-22,1,F,2156-03-09,2222-02-22,66,2222-03-13,2222-02-22,2224-04-08,NaT,NaN,NaN,not synthesized,4,2,0,1,200.0
3,3,0,2-2222-02-22,1,F,2156-03-09,2222-02-22,66,2222-03-13,2222-02-22,2227-03-08,NaT,NaN,NaN,not synthesized,4,2,0,1,200.0
4,4,0,2-2222-02-22,10,M,2156-03-09,2222-02-22,66,NaT,2222-02-22,2222-02-22,2223-08-22,NaN,NaN,not synthesized,4,x,x,1,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79125,79125,0,2-2222-02-22,9997,F,2154-03-10,2222-02-22,68,NaT,2222-02-22,2222-02-28,2223-12-12,NaN,NaN,not synthesized,4,4,0,1,200.0
79126,79126,0,2-2222-02-22,9997,F,2154-03-10,2222-02-22,68,NaT,2222-02-22,2223-07-13,2223-12-12,NaN,NaN,not synthesized,4,4,0,1,200.0
79127,79127,0,2-2222-02-22,9998,M,2156-03-09,2222-02-22,66,NaT,2222-02-22,2222-08-21,2227-09-26,NaN,NaN,not synthesized,4,2,2,1,200.0
79128,79128,0,2-2222-02-22,9998,M,2156-03-09,2222-02-22,66,NaT,2222-02-22,2222-11-26,2227-09-26,NaN,NaN,not synthesized,4,2,2,1,200.0


In [86]:
len(pd.read_excel('/home/dogu86/lung_synthesis/data/raw/LUNG_TRTM_RD.xlsx')['RDT_SITE_CONT'].unique())

2670

In [55]:
import pandas as pd

# Create a sample DataFrame with null values
data = {'A': [1, 2, None, 4, None, 6],
        'B': [None, 2, None, 4, 5, None]}

df = pd.DataFrame(data)

df

,A,B
0,1.0,NaN
1,2.0,2.0
2,NaN,NaN
3,4.0,4.0
4,NaN,5.0
5,6.0,NaN


In [63]:
data

,PT_SBST_NO,CSTR_STRT_YMD,CSTR_CLSF_NM,CSTR_PRPS_CD,CSTR_CYCL_VL,CSTR_END_YMD
1,0,NaT,미수술 항암화학요법,3.0,1,2222-03-25
2,0,2222-03-25,미수술 항암화학요법,3.0,2,2225-06-03
3,0,2225-06-03,미수술 항암화학요법,3.0,3,2225-06-24
4,0,2225-06-24,미수술 항암화학요법,3.0,4,2225-08-05
5,0,2225-08-05,미수술 항암화학요법,3.0,5,2225-08-26
...,...,...,...,...,...,...
425578,9999,2224-09-25,미수술 항암화학요법,3.0,3,2225-02-20
425579,9999,2225-02-20,미수술 항암화학요법,3.0,4,2225-05-15
425580,9999,2225-05-15,미수술 항암화학요법,3.0,5,2225-08-12
425582,9999,2225-08-12,미수술 항암화학요법,3.0,6,2226-01-28


In [122]:
# Define a custom function to calculate and fill null values
def fill_null_with_calculated_value(column):
    result = []
    calculated_value = None

    for i, value in enumerate(column):
        if pd.notna(value):
            result.append(value)
        else:
            calculated_value = column.iloc[i+1] - pd.to_timedelta(28, 'days')
            result.append(calculated_value)
            
            
    return result

In [123]:
data['CSTR_STRT_YMD'] = fill_null_with_calculated_value(data['CSTR_STRT_YMD'])

In [124]:
data

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,CSTR_STRT_YMD,CSTR_SEQ,CSTR_CLSF_NM,CSTR_END_YMD,CSTR_PRPS_CD,CSTR_CYCL_VL,CRTN_DT,CSTR_PRPS_NM
0,NaN,NaN,0,2222-02-22,NaN,미수술 항암화학요법,2222-03-22 00:00:00,3.0,1,NaN,Palliative
1,NaN,NaN,0,2222-03-22,NaN,미수술 항암화학요법,2222-03-25 00:00:00,3.0,2,NaN,Palliative
2,NaN,NaN,0,2222-03-25,NaN,미수술 항암화학요법,2225-06-03 00:00:00,3.0,3,NaN,Palliative
3,NaN,NaN,0,2225-06-03,NaN,미수술 항암화학요법,2225-06-24 00:00:00,3.0,4,NaN,Palliative
4,NaN,NaN,0,2225-06-24,NaN,미수술 항암화학요법,2225-08-05 00:00:00,3.0,5,NaN,Palliative
...,...,...,...,...,...,...,...,...,...,...,...
269760,NaN,NaN,9999,2224-09-25,NaN,미수술 항암화학요법,2225-02-20 00:00:00,3.0,4,NaN,Palliative
269761,NaN,NaN,9999,2225-02-20,NaN,미수술 항암화학요법,2225-05-15 00:00:00,3.0,5,NaN,Palliative
269762,NaN,NaN,9999,2225-05-15,NaN,미수술 항암화학요법,2225-08-12 00:00:00,3.0,6,NaN,Palliative
269763,NaN,NaN,9999,2225-08-12,NaN,미수술 항암화학요법,2226-01-28 00:00:00,3.0,7,NaN,Palliative


In [86]:
data.drop(data[data['CSTR_CYCL_VL']==1].index)

,PT_SBST_NO,CSTR_STRT_YMD,CSTR_CLSF_NM,CSTR_PRPS_CD,CSTR_CYCL_VL,CSTR_END_YMD
2,0,2222-03-25,미수술 항암화학요법,3.0,2,2225-06-03
3,0,2225-06-03,미수술 항암화학요법,3.0,3,2225-06-24
4,0,2225-06-24,미수술 항암화학요법,3.0,4,2225-08-05
5,0,2225-08-05,미수술 항암화학요법,3.0,5,2225-08-26
6,0,2225-08-26,미수술 항암화학요법,3.0,6,2225-09-16
...,...,...,...,...,...,...
425578,9999,2224-09-25,미수술 항암화학요법,3.0,3,2225-02-20
425579,9999,2225-02-20,미수술 항암화학요법,3.0,4,2225-05-15
425580,9999,2225-05-15,미수술 항암화학요법,3.0,5,2225-08-12
425582,9999,2225-08-12,미수술 항암화학요법,3.0,6,2226-01-28


In [82]:
(data[data['CSTR_PRPS_CD']==3.0]).index

Int64Index([     1,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            425565, 425566, 425567, 425571, 425576, 425578, 425579, 425580,
            425582, 425583],
           dtype='int64', length=221155)

In [94]:
data[data['SGPT_PATL_M_STAG_VL']==1]

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,SGPT_ACPT_YMD,SGPT_SEQ,SGPT_READ_YMD,SGPT_LUNGM_SIZE_VL,SGPT_CELL_TYPE_CONT,SGPT_CELL_DIFF_CONT,SGPT_VPI_CONT,...,SGPT_SFMG_VL,SGPT_LN_DSCT_CONT,SGPT_MTST_LN_CNT,SGPT_MTST_LMP_SITE_CONT,SGPT_PATL_STAG_VL,SGPT_PATL_ETC_STAG_VL,SGPT_PATL_T_STAG_VL,SGPT_PATL_N_STAG_VL,SGPT_PATL_M_STAG_VL,CRTN_DT
4929,NaN,NaN,10235,2225-09-02 00:00:00,NaN,2225-09-05 00:00:00,2.2,Non small cell carcinoma,NaN,Involved,...,7cm,NaN,18.0,NaN,NaN,NaN,2.0,2,1.0,NaN
21775,NaN,NaN,1121,2225-09-02 00:00:00,NaN,2225-09-05 00:00:00,2.2,Non small cell carcinoma,NaN,Involved,...,7cm,NaN,18.0,NaN,NaN,NaN,2.0,2,1.0,NaN
33077,NaN,NaN,12036,2222-03-18 00:00:00,NaN,2222-03-21 00:00:00,4.5,Non small cell carcinoma,MD,Not involved,...,2.5cm,NaN,2.0,NaN,NaN,NaN,2.0,1,1.0,NaN
36327,NaN,NaN,12299,2222-02-22 00:00:00,NaN,2222-02-25 00:00:00,2.9,Non small cell carcinoma,MD,Not involved,...,4.5cm,NaN,1.0,NaN,NaN,NaN,1.0,1,1.0,NaN
40964,NaN,NaN,12707,2223-06-04 00:00:00,NaN,2223-06-07 00:00:00,2.3,Non small cell carcinoma,NaN,Not involved,...,2.1cm,NaN,2.0,NaN,NaN,NaN,1.0,1,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406951,NaN,NaN,8488,2222-02-22 00:00:00,NaN,2222-02-25 00:00:00,2.8,Non small cell carcinoma,NaN,Not involved,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2,1.0,NaN
411269,NaN,NaN,9051,2222-08-20 00:00:00,NaN,2222-08-23 00:00:00,2.1,Non small cell carcinoma,NaN,Not involved,...,1.2cm,NaN,3.0,NaN,NaN,NaN,1.0,2,1.0,NaN
411636,NaN,NaN,9102,2222-03-28 00:00:00,NaN,2222-03-31 00:00:00,0.9,Non small cell carcinoma,NaN,Not involved,...,0.4cm,NaN,0.0,NaN,NaN,NaN,4.0,0.0,1.0,NaN
415965,NaN,NaN,9568,2223-10-30 00:00:00,NaN,2223-11-02 00:00:00,4.1,Non small cell carcinoma,NaN,Involved,...,0.1cm,NaN,7.0,NaN,NaN,NaN,3.0,2,1.0,NaN


In [110]:
data[data['OPRT_LUCN_OPRT_KIND_CD']==99].index

Int64Index([   53,   150,   219,   320,   418,   472,   534,   601,   645,
              683,   827,   908,  1011,  1020,  1315,  1327,  1464,  1522,
             1678,  2529,  2620,  2789,  2898,  2952,  3012,  3079,  3119,
             3160,  3298,  3376,  3482,  3491,  3787,  3799,  3991,  4066,
             4162,  5016,  5109,  5277,  5379,  5434,  5497,  5567,  5580,
             5610,  5650,  5796,  5879,  5991,  6001,  6070,  6305,  6316,
             6516,  6684,  6732,  7564,  7606,  7666,  7841,  7897,  7949,
             8005,  8069,  8136,  8176,  8219,  8316,  8375,  8453,  8561,
             8572,  8866,  8876,  9072,  9236,  9730,  9808,  9910,  9919,
            10225, 10236, 10433, 10597],
           dtype='int64')

In [115]:
file_name = "LUNG_TRTM_CASB"
data = load_file_epsilon(1000)
data = data.rename(columns = {'TIME':'CSTR_STRT_YMD'})
data = data.dropna(subset=['CSTR_CLSF_NM', 'CSTR_PRPS_CD'], how='any')
data['CSTR_END_YMD'] = data['CSTR_STRT_YMD'] + pd.to_timedelta(28, 'days') # need to check quantity of additonal
data['CSTR_CYCL_VL'] = data.groupby(['PT_SBST_NO','CSTR_PRPS_CD']).cumcount()+1

grouped = data.groupby(['PT_SBST_NO', 'CSTR_PRPS_CD'])
data['CSTR_STRT_YMD'] = grouped['CSTR_END_YMD'].shift(1)
data['CSTR_STRT_YMD'] = grouped['CSTR_STRT_YMD'].apply(lambda x: x.fillna('2222-02-22'))

original_path = PROJ_PATH.joinpath(f'data/raw/{file_name}.xlsx')
original_data = pd.read_excel(original_path)
head = pd.read_excel(original_path, nrows=0)
data = data.drop(data[data['CSTR_PRPS_CD']==0].index)
data = data.drop(data[data['CSTR_PRPS_CD']==121].index)
data = pd.concat([head, data])

reference1 = original_data[['CSTR_PRPS_CD','CSTR_PRPS_NM']].drop_duplicates().reset_index(drop=True)
data = data.drop(columns='CSTR_PRPS_NM')
data = data.merge(reference1, how='left', on='CSTR_PRPS_CD')

In [116]:
data

,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,CSTR_STRT_YMD,CSTR_SEQ,CSTR_CLSF_NM,CSTR_END_YMD,CSTR_PRPS_CD,CSTR_CYCL_VL,CRTN_DT,CSTR_PRPS_NM
0,NaN,NaN,0,2222-02-22 00:00:00,NaN,미수술 항암화학요법,2222-03-22 00:00:00,3.0,1,NaN,Palliative
1,NaN,NaN,0,2222-03-22 00:00:00,NaN,미수술 항암화학요법,2222-03-25 00:00:00,3.0,2,NaN,Palliative
2,NaN,NaN,0,2222-03-25 00:00:00,NaN,미수술 항암화학요법,2225-06-03 00:00:00,3.0,3,NaN,Palliative
3,NaN,NaN,0,2225-06-03 00:00:00,NaN,미수술 항암화학요법,2225-06-24 00:00:00,3.0,4,NaN,Palliative
4,NaN,NaN,0,2225-06-24 00:00:00,NaN,미수술 항암화학요법,2225-08-05 00:00:00,3.0,5,NaN,Palliative
...,...,...,...,...,...,...,...,...,...,...,...
269760,NaN,NaN,9999,2224-09-25 00:00:00,NaN,미수술 항암화학요법,2225-02-20 00:00:00,3.0,4,NaN,Palliative
269761,NaN,NaN,9999,2225-02-20 00:00:00,NaN,미수술 항암화학요법,2225-05-15 00:00:00,3.0,5,NaN,Palliative
269762,NaN,NaN,9999,2225-05-15 00:00:00,NaN,미수술 항암화학요법,2225-08-12 00:00:00,3.0,6,NaN,Palliative
269763,NaN,NaN,9999,2225-08-12 00:00:00,NaN,미수술 항암화학요법,2226-01-28 00:00:00,3.0,7,NaN,Palliative


In [117]:
regn = pd.read_excel(f'/mnt/synthetic_data/lung/data/processed/3_postprocess/LUNG_TRTM_CASB_{1000}.xlsx')
regn['PT_SBST_NO'] = regn['PT_SBST_NO'].astype('str')
regn

,Unnamed: 0,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,CSTR_STRT_YMD,CSTR_SEQ,CSTR_CLSF_NM,CSTR_END_YMD,CSTR_PRPS_CD,CSTR_PRPS_NM,CSTR_CYCL_VL,CRTN_DT
0,0,0,2-2222-02-22,0,2222-02-22,1,미수술 항암화학요법,2222-03-22,3,Palliative,1,2023-10-20
1,1,0,2-2222-02-22,0,2222-03-22,1,미수술 항암화학요법,2222-03-25,3,Palliative,2,2023-10-20
2,2,0,2-2222-02-22,0,2222-03-25,1,미수술 항암화학요법,2225-06-03,3,Palliative,3,2023-10-20
3,3,0,2-2222-02-22,0,2225-06-03,1,미수술 항암화학요법,2225-06-24,3,Palliative,4,2023-10-20
4,4,0,2-2222-02-22,0,2225-06-24,1,미수술 항암화학요법,2225-08-05,3,Palliative,5,2023-10-20
...,...,...,...,...,...,...,...,...,...,...,...,...
269760,269760,0,2-2222-02-22,9999,2224-09-25,1,미수술 항암화학요법,2225-02-20,3,Palliative,4,2023-10-20
269761,269761,0,2-2222-02-22,9999,2225-02-20,1,미수술 항암화학요법,2225-05-15,3,Palliative,5,2023-10-20
269762,269762,0,2-2222-02-22,9999,2225-05-15,1,미수술 항암화학요법,2225-08-12,3,Palliative,6,2023-10-20
269763,269763,0,2-2222-02-22,9999,2225-08-12,1,미수술 항암화학요법,2226-01-28,3,Palliative,7,2023-10-20


In [ ]:
regn_info = []
for i in regn['PT_SBST_NO'].unique():
    grouped = regn[regn['CSTR_CYCL_VL'] == 1].groupby(by='PT_SBST_NO').get_group(i)
    start = grouped['CSTR_STRT_YMD'].iloc[0]
    end = grouped['CSTR_END_YMD'].iloc[-1]
    regn_info.append([i,start,end])

regn_info_pd = pd.DataFrame(regn_info,columns=['PT_SBST_NO','CSTR_STRT_YMD','CSTR_END_YMD'])

In [126]:
start = regn[regn['CSTR_CYCL_VL'] == 1].groupby(by='PT_SBST_NO').get_group('9997')['CSTR_STRT_YMD'].iloc[0]

In [131]:
regn.groupby(by='PT_SBST_NO').get_group('1000')

,Unnamed: 0,CENTER_CD,IRB_APRV_NO,PT_SBST_NO,CSTR_STRT_YMD,CSTR_SEQ,CSTR_CLSF_NM,CSTR_END_YMD,CSTR_PRPS_CD,CSTR_PRPS_NM,CSTR_CYCL_VL,CRTN_DT
162,162,0,2-2222-02-22,1000,2222-02-22,1,미수술 항암화학요법,2222-03-22,3,Palliative,1,2023-10-20
163,163,0,2-2222-02-22,1000,2222-03-22,1,미수술 항암화학요법,2222-05-01,3,Palliative,2,2023-10-20
164,164,0,2-2222-02-22,1000,2222-05-01,1,미수술 항암화학요법,2222-05-22,3,Palliative,3,2023-10-20
165,165,0,2-2222-02-22,1000,2222-05-22,1,미수술 항암화학요법,2222-06-12,3,Palliative,4,2023-10-20
166,166,0,2-2222-02-22,1000,2222-06-12,1,미수술 항암화학요법,2222-07-03,3,Palliative,5,2023-10-20
167,167,0,2-2222-02-22,1000,2222-07-03,1,미수술 항암화학요법,2222-07-24,3,Palliative,6,2023-10-20
168,168,0,2-2222-02-22,1000,2222-07-24,1,미수술 항암화학요법,2222-08-14,3,Palliative,7,2023-10-20
169,169,0,2-2222-02-22,1000,2222-08-14,1,미수술 항암화학요법,2222-08-21,3,Palliative,8,2023-10-20
170,170,0,2-2222-02-22,1000,2222-08-21,1,미수술 항암화학요법,2222-09-04,3,Palliative,9,2023-10-20
171,171,0,2-2222-02-22,1000,2222-09-04,1,미수술 항암화학요법,2222-09-25,3,Palliative,10,2023-10-20


In [ ]:
grouped = regn[regn['CSTR_CYCL_VL'] == 1].groupby(by='PT_SBST_NO').get_group(i)